# НАЧАЛО ГЛАВЫ 6: ГЛУБОКОЕ ОБУЧЕНИЕ ДЛЯ ТЕКСТА И ПОСЛЕДОВАТЕЛЬНОСТЕЙ

# One-hot encoding (прямое кодирование)

Прямое кодирование на уровне слов

In [ ]:
# прямое кодирование (one-hot encoding) делается в общем так:
# - каждому слову текста присваивается номер i
# - создается вектор размером с длину алфавита, из которого состоят эти слова
# - i-ый элемент этого вектора = 1, остальные = 0

import numpy as np

# один элемент = один образец
samples = ['The cat sat on the mat', 'The dog ate my homework']

# словарь token - его индекс
token_index = {}
i = 0
for sample in samples:
  for word in sample.split():
    if word not in token_index:
      token_index[word] = i
      i += 1

# максимальное шифруемое количество слов
max_length = 10

# трехмерный тензор, где каждый двумерный тензор = слово
# .values() = [1,2,3,4,5,6,7,8,9,10]
results = np.zeros(shape=(len(samples), max_length, max(token_index.values()) + 1))

for i, sample in enumerate(samples):
  for j, word in enumerate(sample.split()):
    # получение индекса слова в предложении
    index = token_index.get(word)
    results[i, j, index] = 1

for el in results:
  print(el)


Прямое кодирование на уровне символов

In [ ]:
import string

samples = ['The cat sat on the mat', 'The dog ate my homework']
# все существующие символы ASCII
characters = string.printable
token_index = {}
for j, character in enumerate(characters):
  token_index[character] = j

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))

for i, sample in enumerate(samples):
  for j, character in enumerate(sample):
    index = token_index.get(character)
    results[i, j, index] = 1


Прямое кодирование слов с помощью Keras

In [ ]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat', 'The dog ate my homework', 'zhopa']

# такой токенизатор сможет закодировать 1000 слов
tokenizer = Tokenizer(num_words=1000)
# создание индексов всех слов
tokenizer.fit_on_texts(samples)
# преобразование слов в списке целочисленных индексов
tokenizer.texts_to_sequences(samples)
# получение прямого кодирование one-hot
# матрица формы (len(samples), num_words)
# каждый i-ый элемент - один образец (предложение или слово - не важно)
one_hot_code = tokenizer.texts_to_matrix(samples, mode='binary')
print(f'Found {tokenizer.word_index} unique tokens')

for i, sample in enumerate(samples):
  print(f'Encoded {sample} is: \n {one_hot_code[i]}')


Прямое кодирование на уровне слов у использованием хэширования

In [ ]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# слова будут сохранятся как тензоры длины 1000
# чем ближе количество слов к 1000, тем больше хэш-коллизий - когда два
# слова получают одинаковых хэш-код и сетка не может их различить
dims = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dims))
for i, sample in enumerate(samples):
  for j, word in enumerate(sample.split()):
    # встроенная функция hash возвращает хэш-код слова или символа
    index = abs(hash(word)) % dims
    print(f'index is {index}')
    results[i, j, index] = 1

for i, sample in enumerate(samples):
  for j, word in enumerate(sample.split()):
    print(f'Encoded {word} is {results[i][j]}')
    

# Векторное представление слов (часть 1)


In [ ]:
# прямое кодирование слов дает многоразмерные разреженные тензоры
# есть векторное представление слов
# векторное представление конструируется на основе даннх, имеет меньшую размерность
# синонимы представляются схожими векторами
# расстояние между векторами в n-мерном пространстве показывает схожесть их семантики
# пусть на двумерной плоскости есть две точни (собака) (кошка), тогда вектор 
# (собака)-(кошка) имеет логический смысл - переход от псовых к кошачим
# универсального векторного пространства для всего человеческого языка НЕТ
# для каждой новой задачи нужно обучать новое пространство

In [ ]:
from keras.layers import Embedding

# Embedding принимает минимум два аргумента: количество токенов и размерность
# 1000 слов, каждое = вектор длины 64
# Embedding работает как словарь. Он принимает на вход целое число,
# ищет его во внутреннем словаре по ключам. Возвращает значение - тензор
# ПРИНИМАЕТ тензор (образцы, длина последовательности)
# ВОЗВРАЩАЕТ тензор (образцы, длина последовательности, размерность_векторного представления)
embedding_layer = Embedding(1000, 64)
